In [55]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings



In [56]:
df = pd.read_csv("data\StudentsPerformance.csv")

In [57]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


preparing x and y variable 


In [58]:
x = df.drop(columns=['math score'],axis=1)

In [59]:
y=df['math score']

In [60]:
x

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [61]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [62]:
num_features = x.select_dtypes(exclude="object").columns
cat_features = x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor =ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScalar",numeric_transformer,num_features),
        

    ]



)

In [63]:
x = preprocessor.fit_transform(x)


In [64]:
x.shape

(1000, 19)

In [65]:
#separate the data set into train and test 

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =42)
x_train.shape , x_test.shape

((800, 19), (200, 19))

In [66]:
def evaluate_model (true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error (true, predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)

    return mae , rmse , r2_square

In [67]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for model_name, model in models.items():

    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(model_name)
    model_list.append(model_name)

    print("Model performance for training set")
    print("- RMSE : {:.4f}".format(model_train_rmse))
    print("- MAE  : {:.4f}".format(model_train_mae))
    print("- R2   : {:.4f}".format(model_train_r2))

    print("---------------------------------------------------")

    print("Model performance for test set")
    print("- RMSE : {:.4f}".format(model_test_rmse))
    print("- MAE  : {:.4f}".format(model_test_mae))
    print("- R2   : {:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)
    


Linear Regression
Model performance for training set
- RMSE : 5.3274
- MAE  : 4.2788
- R2   : 0.8741
---------------------------------------------------
Model performance for test set
- RMSE : 5.4096
- MAE  : 4.2259
- R2   : 0.8797
Lasso
Model performance for training set
- RMSE : 6.5938
- MAE  : 5.2063
- R2   : 0.8071
---------------------------------------------------
Model performance for test set
- RMSE : 6.5197
- MAE  : 5.1579
- R2   : 0.8253
Ridge
Model performance for training set
- RMSE : 5.3233
- MAE  : 4.2650
- R2   : 0.8743
---------------------------------------------------
Model performance for test set
- RMSE : 5.3904
- MAE  : 4.2111
- R2   : 0.8806
K-Neighbors Regressor
Model performance for training set
- RMSE : 5.7122
- MAE  : 4.5187
- R2   : 0.8553
---------------------------------------------------
Model performance for test set
- RMSE : 7.2516
- MAE  : 5.6160
- R2   : 0.7839
Decision Tree
Model performance for training set
- RMSE : 0.2795
- MAE  : 0.0187
- R2   : 0.

# RESULTS 

In [68]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name','r2_score']).sort_values(by = ["r2_score"],ascending=False)

,Model Name,r2_score
2,Ridge,0.880593
0,Linear Regression,0.879740
5,Random Forest Regressor,0.852932
7,CatBoost Regressor,0.851632
8,AdaBoost Regressor,0.849198
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbors Regressor,0.783898
4,Decision Tree,0.739026
